# Importing Libraries

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup 
from bs4 import BeautifulSoup 
import json
import requests
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import time
import datetime
import os.path
import math
import re
from IPython.display import Image
from IPython.core.display import HTML 
import sys
import warnings


from selenium.webdriver.support.ui import WebDriverWait     
from selenium.webdriver.common.by import By     
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [9]:
#if not sys.warnoptions:
#    warnings.simplefilter("ignore")
    
#warnings.filterwarnings("ignore", category=FutureWarning)

np.seterr(all='warn')
#warnings.simplefilter("always")
#warnings.filterwarnings("ignore", category=FutureWarning)

{'divide': 'warn', 'over': 'warn', 'under': 'warn', 'invalid': 'warn'}

# Parameters

In [ ]:
users_lists_path = path + 'users_lists/'

users_file = path + 'users'
users_ext_file = path + 'users_ext'

relationstrk_file = path + 'relationstrk'
likestrk_file = path + 'likestrk'
relations_file = path + 'relations'
relations_ext_file = path + 'relations_ext'

vipuserstrk_file = path + 'vipuserstrk'

status_file = path + 'status'
status_likes_file = path + 'status_likes'

pwd_file = path + 'pwd'


user_info_ext_file = path + 'user_info_ext'


In [5]:
usercolumntypes = {   'userid': object
                    , 'username': object
                    , 'full_name': object
                    , 'profile_pic_url': object
                    , 'is_private': bool
                    , 'is_verified': bool
                    , 'last_update_time_basic': str
                    , 'is_business_account': object
                    , 'posts': float
                    , 'followers': float
                    , 'followed': float
                    , 'highlighted_stories': float
                    , 'igtv': float
                    , 'collections': float
                    , 'connected_fb_page': float
                    , 'biography': object
                    , 'is_joined_recently': object
                    , 'business_category_name': object
                    , 'overall_category_name': float
                    , 'category_enum': object
                    , 'has_ar_effects': object
                    , 'has_clips': object
                    , 'has_guides': object
                    , 'has_channel': object
                    , 'external_url': object
                    , 'profile_pic_url_hd': object
                    , 'external_url_linkshimmed': object
                    , 'last_update_time_info': str
                  }

#para hacer esto con otras tablas ayuda mucho esto (ojo! no usar ):
#users.dtypes

# Screenshot function

In [6]:
## Defining screenshot function
def screenCap():
    millis = int(round(time.time() * 1000))
    imgName=millis
    driver.save_screenshot('Screens/'+"test"+str(imgName)+".png")
    img=mpimg.imread('Screens/'+"test"+str(imgName)+".png")
    imgplot = plt.imshow(img)
    plt.show()

# Login function

In [1]:

def do_Login(username,password):
    driver.get('https://www.instagram.com/accounts/login/')
    time.sleep(1)
    driver.find_element_by_xpath("//*[text()='Aceptar']").click()
    driver.find_element_by_name("username").send_keys(username)
    driver.find_element_by_name("password").send_keys(password)
    time.sleep(1)
    driver.find_element_by_xpath("//*[text()='Iniciar sesión']").click()
    time.sleep(5)
    driver.find_element_by_xpath("//*[text()='Ahora no']").click()
    time.sleep(5)
    driver.find_element_by_xpath("//*[text()='Ahora no']").click()

    #screenCap()

# Open IG function

In [8]:
def searchAndPrint(searchText):
    driver.find_element_by_xpath("//*[@placeholder='Busca']").clear()
    driver.find_element_by_xpath("//*[@placeholder='Busca']").send_keys(searchText)
    #screenCap()
    time.sleep(3)
    ele=driver.find_element_by_xpath("//*[@class='fuqBx']")
    results=ele.find_elements_by_tag_name("a")
    l=[]
    for i in results:
        #Excluding Hashtags
        if "#" not in i.text:
            l.append(i.text.partition('\n')[0])
            print(i.text.partition('\n')[0],end=" , ")
    return l

# Search and Open Profile Function

In [9]:
def searchAndOpenProfile(profileName):
    driver.find_element_by_xpath("//*[@placeholder='Busca']").clear()
    driver.find_element_by_xpath("//*[@placeholder='Busca']").send_keys(profileName)
    time.sleep(3)
    #screenCap()
    ele=driver.find_element_by_xpath("//*[@class='fuqBx']")
    results=ele.find_elements_by_tag_name("a")
    for i in results:
        if "#" not in i.text:
            i.click()
            #print(i.text.partition('\n')[0]+" Profile Opened")
            #plt.show()
            #driver.switch_to.default_content().click()
            #driver..click()
            #ActionChains(driver).move_to_element(driver.find_element_by_xpath("//div[@class='aIYm8 coreSpriteSearchClear']")).click().perform()
            driver.find_element_by_xpath("//*[@class='aIYm8 coreSpriteSearchClear']").click()
            break;

# Extract Profile Stats (Selenium)

In [40]:
def profileBasicInfo(profileName):
    #searchAndOpenProfile(profileName)
    url = 'https://www.instagram.com/' + profileName
    driver.get(url)
    time.sleep(2)
    ele=driver.find_element_by_xpath("//*[@class='k9GMp ']")
    head=ele.find_elements_by_tag_name("span")
    posts = int(head[1].text.replace(".","")) 
    followers_rounded = head[2].text
    followers_exact = head[2].get_attribute('title') 
    if followers_exact == '':
        account_type = 'private'
        k = 1 #the structure is a litle bit different
    else:
        account_type = 'public'
        k = 0
    followers_rounded = head[2+k].text
    followers_exact = int(head[2+k].get_attribute('title').replace(".","")) 
    followed = int(head[3+2*k].text.replace(".",""))
    
    #convert followers rounded into a number
    multiply = 1
    if followers_rounded.find("k") >= 0:
        multiply = 1000
    elif followers_rounded.find("mm") >=0:
        multiply = 1000000
    followers_rounded = int(float(re.sub('[a-z]','', followers_rounded).replace('.','').replace(',','.'))*multiply)
    
    #Print results
    print('Stats of ' + profileName + ' (' + account_type + ' account)')
    print('----------------------------------------------')
    print('posts: ' + str(posts))
    if account_type == 'private' and multiply > 1:
        print('followers: ' + str(followers_rounded) + ' (cannot know exact number)')
    elif multiply == 1:
        print('followers: ' + str(followers_rounded))
    else:
        print('followers: ' + str(followers_rounded) + ' (exactly ' + str(followers_exact) +')')
    print('following: ' + str(followed))
    
    #build output dataframe
    data = {'username': [profileName],
            'posts': [posts], 
            'followers_rounded': [followers_rounded],
            'followers_exact': [followers_exact],
            'followed': [followed]
           } 
    df = pd.DataFrame(data) 
    return df

#
#'-nal3 '

# User Info (Soap)

In [ ]:
def UserInfo(username):

    url = 'https://www.instagram.com/' + username + '/?__a=1'
    edge = 'user'

    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    s = json.loads(soup.text)
    #user_info = pd.json_normalize(s)
    user_info = pd.json_normalize(s["graphql"][edge])

    user_info = user_info[['id',  'username', "full_name",  'is_private',  'is_verified', 'is_business_account',
                             'edge_owner_to_timeline_media.count', 'edge_followed_by.count', 'edge_follow.count', 
                             'highlight_reel_count',  'edge_felix_video_timeline.count', 'edge_media_collections.count',
                             'connected_fb_page', "biography", 'is_joined_recently', 'business_category_name',
                             'overall_category_name', 'category_enum', 
                             'has_ar_effects', 'has_clips', 'has_guides', 'has_channel',
                             'external_url', 'profile_pic_url', 'profile_pic_url_hd', 'external_url_linkshimmed'
                            ]]
    
    user_info = user_info.rename(columns = {'id': 'userid',
                                            'edge_owner_to_timeline_media.count': 'posts',
                                            'edge_followed_by.count': 'followers',
                                            'edge_follow.count': 'followed',                                       
                                            'highlight_reel_count': 'highlighted_stories',
                                            'edge_felix_video_timeline.count': 'igtv',
                                            'edge_media_collections.count': 'collections'
                                           })
    
    now = datetime.datetime.now()
    user_info["inserted_time"] = now
    user_info["last_update_time"] = now
    
    #Update users table
    if os.path.isfile(user_info_ext_file):
        user_info.to_csv(user_info_ext_file, mode='a', header=False, index = False) #append
    else:
        user_info.to_csv(user_info_ext_file, mode='w', header=True , index = False) #create/overwrite
        
    updateUsersTable(user_info_ext_file, users_file)
    
    user_info = user_info.drop(columns = ["last_update_time"])
    
    #Save tracking of this user in vipuserstrk
    UpdateVipUsers(user_info)
    
    return user_info

# Extract followers of user (Selenium- slow)


In [11]:
def followersList(handleName,numberOfFollowers):
    searchAndOpenProfile(handleName)
    time.sleep(3)
    handle='/'+handleName+'/followers/'
    # Making dynamic xpath
    xpath="//*[@href='"+handle+"'"+']' 
    driver.find_element_by_xpath(xpath).click()
    time.sleep(2)
    i=0
    j=0
    while i<=numberOfFollowers+j:
        time.sleep(0.5)
        element=driver.find_element_by_xpath("//*[@class='PZuss']")
        results=element.find_elements_by_tag_name("a")
        
        x=[]
        for k in results:
            if k.text.strip()!='':
                x.append(k.text.strip())
        
        if (len(x)<numberOfFollowers):
            followerWindow = driver.find_element_by_xpath("//*[@class='isgrP']")
            elements=followerWindow.find_elements_by_tag_name("a")
            driver.execute_script("arguments[0].scrollIntoView();", elements[i])
            i=i+1
            j=j+1
        else:
        
            element_new=driver.find_element_by_xpath("//*[@class='PZuss']")
            results_new=element.find_elements_by_tag_name("a")
            l=[]
            count=0
            for r in results_new:
                if r.text.strip()!='':
                    count=count+1
                    l.append(r.text.strip())
                    if count==numberOfFollowers:
                        #driver.find_element_by_xpath("//*[@class='wpO6b ']").click()
                        driver.get('https://www.instagram.com/'+ handleName)
                        return l
                        break
            print(l)
            driver.get('https://www.instagram.com/'+ handleName)
            return l
            break


# Read Followers of user (limit > 20k)

In [7]:
def readFollowers(identificator, process, max_status_hours, requests, risk_level):
    #max_status_hours: maximum hours to look backwards in status log table
    #process can be followers or followed or hashtags
    
    #user data
    if process == 'followers' or process == 'followed':
        username = identificator
        user_info = UserInfo(username)
        userid = user_info.userid[0]
        identificatorvar = 'username'
        identificatorid = userid
        urlidvar = 'id'
        
    #Parameters that depend on the process    
    if process == 'hashtags':
        query_hash = '???'
        edge = '???'
    elif process == 'likes':
        status_file_local = status_likes_file
        relationstrk_file_local = likestrk_file
        query_hash = 'd5d763b1e2acf209d62d22d184488e57'
        edge = 'edge_liked_by'
        edge_parent = 'shortcode_media'
        identificatorvar = 'shortcode'
        identificatorid = identificator
        urlidvar = 'shortcode'
        instances = 1000
    elif process == 'followers':
        status_file_local = status_file
        relationstrk_file_local = relationstrk_file
        query_hash = 'c76146de99bb02f6415203be841dd25a'
        edge = 'edge_followed_by'
        edge_parent = 'user'
        instances = user_info.followers[0]
    else: #'followed'
        status_file_local = status_file
        relationstrk_file_local = relationstrk_file
        query_hash = 'd04b0a864b4b54837c0d870b0e77e076'
        edge = 'edge_follow'
        edge_parent = 'user'
        instances = user_info.followed[0]
        process = 'followed'

    process_name = process + "_extraction"
        
    #parameters
    
    sr = 1  #sleep ratio
    #modify sleep ratio as a function of risk_level (which should be between 0 and 1)
    if risk_level > 1: 
        sr = 0.5
    elif risk_level < 0:
        sr = 2
    else:
        sr = 2 - 1.5*risk_level
    if instances > 10000:
        sr = 2*sr
    print("Sleep ratio: " + str(round(sr,2)))
    
    max_retries = 5           # if gets rate limit  
    results_num = 50          # how many users requests in each bulk (admits max 50 but sometimes downloads less ~48)
    short_sleep = 2*sr        # every bulk
    medium_sleep = 5*sr       # every 10 bulks
    long_sleep = 150*sr       # every 200 bulks or too_many_requests_message
    atpf = 48/short_sleep     # atpf = average time per follower
    
    #initialize void data frames
    users = pd.DataFrame([])
    users_ext = pd.DataFrame([])
    relations = pd.DataFrame([])
    relations_ext = pd.DataFrame([])
    
    #first url
    url = 'https://www.instagram.com/graphql/query/?query_hash=' + query_hash + '&variables=%7B%22id%22%3A%22' + str(identificatorid) + '%22%2C%22include_reel%22%3Atrue%2C%22fetch_mutual%22%3Afalse%2C%22first%22%3A'+ str(results_num) + '%7D'
    url = 'https://www.instagram.com/graphql/query/?query_hash=' + query_hash + '&variables=%7B%22' + urlidvar + '%22%3A%22' + str(identificatorid) + '%22%2C%22include_reel%22%3Atrue%2C%22fetch_mutual%22%3Afalse%2C%22first%22%3A'+ str(results_num) + '%7D'

    ins_rem = instances  #instances remaining 
    ins_ext = 0          #instances extracted 
    
    #Load last status
    if os.path.isfile(status_file_local):
        status = pd.read_csv(status_file_local)
        print('Loaded file' + status_file_local)
        #convert inserted_time to datetime type, since read_csv reads as string
        status['inserted_time'] =  status['inserted_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))
        #keep only last max_status_hours & only desired process & user analyzed
        status = status[status.inserted_time > datetime.datetime.now() - datetime.timedelta(hours=max_status_hours)]
        status = status[status.process == process_name]
        status = status[status[identificatorvar] == identificator]
        if len(status) > 0:
            status = status[status.inserted_time == max(status.inserted_time)]
            if not(status.is_finished.iloc[0]):
                url = status.url.iloc[0]
                ins_ext = int(round(instances*status.percent_done.iloc[0]/100,0))  # extracted followers
                ins_rem  = instances - ins_ext # remaining followers
                print('Taking last status... ' + str(round(status.percent_done.iloc[0]))+ "% already extracted. Let's continue.")
            else:
                print(identificator + ' already extracted in the last ' + str(max_status_hours) + ' hours')
                return 0 

    #Print time aproximation
    print('\nExtracting ' + str(ins_rem) + ' ' + process + ' of ' + identificator + ' (total: ' + str(instances) + 
          ') will take ' + str(math.floor(ins_rem/(atpf*60))) + ' minutes ' + str(int((ins_rem/atpf % 60))) + ' seconds aproximately.')
    
    
    #Initialize some variables before loop
    has_next_page = True
    
    print('Extracted: ' + str(ins_ext), end='')


    while has_next_page:
    
        #Go to url and extract text
        time.sleep(short_sleep)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        s = json.loads(soup.text)
        
        #rate limit control retry
        retries = 1
        while "message" in s and retries <= max_retries:
            print('\n' + s["message"])
            time.sleep(long_sleep*retries)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            s = json.loads(soup.text)
            retries = retries + 1
        
        #get next iterarion params
        has_next_page = s["data"][edge_parent][edge]["page_info"]["has_next_page"]
        end_cursor = s["data"][edge_parent][edge]["page_info"]["end_cursor"]
        if has_next_page:
            #url = 'https://www.instagram.com/graphql/query/?query_hash=' + query_hash + '&variables=%7B%22id%22%3A%22' + str(userid) + '%22%2C%22include_reel%22%3Atrue%2C%22fetch_mutual%22%3Afalse%2C%22first%22%3A' + str(results_num) + '%2C%22after%22%3A%22' + end_cursor + '%22%7D'
            url = 'https://www.instagram.com/graphql/query/?query_hash=' + query_hash + '&variables=%7B%22' + urlidvar + '%22%3A%22' + str(identificatorid) + '%22%2C%22include_reel%22%3Atrue%2C%22fetch_mutual%22%3Afalse%2C%22first%22%3A'+ str(results_num) + '%2C%22after%22%3A%22' + end_cursor + '%22%7D'
        #download
        if s["data"][edge_parent][edge]["edges"] != []:
            users_ext = pd.json_normalize(s["data"][edge_parent][edge]["edges"])
            colnames = list(users_ext.columns)
            colnames = [sub.replace('node.', '') for sub in colnames] 
            colnames = [sub.replace('.', '_') for sub in colnames] 
            users_ext.columns = colnames
            now = datetime.datetime.now()
            users_ext.loc[:,"inserted_time"] = now
            users_ext.loc[:,"last_update_time"] = now
            users_ext = users_ext[["id", "username", "full_name", "profile_pic_url", "is_private", "is_verified", "inserted_time","last_update_time"]]
            users_ext.rename(columns = {'id': 'userid'}, inplace = True)

            #save users_ext_file
            if os.path.isfile(users_ext_file):
                users_ext.to_csv(users_ext_file, mode='a', header=False, index = False) #append
            else:
                users_ext.to_csv(users_ext_file, mode='w', header=True, index = False) #create/overwrite

            #UsersRelations
            relations_ext = users_ext[["userid", "username", "inserted_time","last_update_time"]]
            relations_ext.loc[:,"process"] = process_name
            if process == 'likes':
                relations_ext.loc[:,'shortcode'] = identificator
                relations_ext.loc[:,"status"] = "likes"
                relations_ext = relations_ext[["status", "shortcode", "userid", "username","inserted_time","process"]]
            elif process == 'followers':
                relations_ext = relations_ext.rename(columns = {'userid': 'followed_by_userid'})
                relations_ext = relations_ext.rename(columns = {'username': 'followed_by_username'})
                relations_ext.loc[:,"userid"] = userid
                relations_ext.loc[:,"username"] = username
                relations_ext.loc[:,"status"] = "following"
                relations_ext = relations_ext[["status","userid", "username","followed_by_userid","followed_by_username","inserted_time","process"]]
            elif process == 'followed':
                relations_ext.loc[:,'followed_by_userid'] = userid
                relations_ext.loc[:,'followed_by_username'] = username
                relations_ext.loc[:,"status"] = "following"
                relations_ext = relations_ext[["status","userid", "username","followed_by_userid","followed_by_username","inserted_time","process"]]

            #save users_ext_file
            if os.path.isfile(relationstrk_file_local):
                relations_ext.to_csv(relationstrk_file_local, mode='a', header=False, index = False) #append
            else:
                relations_ext.to_csv(relationstrk_file_local, mode='w', header=True, index = False) #create

            #concat dfs
            relations = pd.concat([relations_ext, relations], ignore_index=True)

        #save status
        now = datetime.datetime.now()
        if process == 'followers' or process == 'followed':
            d = {'userid': [userid], 'username': [username], 'process': [process_name], 'is_finished':[not(has_next_page)],
                 'percent_done': [100*(ins_ext + len(relations.index))/instances], 'inserted_time': [now],
                 'inserted_date': [now.date()], 'end_cursor':[ end_cursor], 'url': [url]}
        elif process == 'likes':
            d = {'shortcode': [identificator], 'process': [process_name], 'is_finished':[not(has_next_page)],
                 'percent_done': [100*(ins_ext + len(relations.index))/instances], 'inserted_time': [now],
                 'inserted_date': [now.date()], 'end_cursor':[ end_cursor], 'url': [url]}
        status = pd.DataFrame(data=d) 
        if os.path.isfile(status_file_local):
            status.to_csv(status_file_local, mode='a', header=False, index = False) #append
        else:
            status.to_csv(status_file_local, mode='w', header=True, index = False) #create/overwrite
        
        #print users extracted
        print(' -> ' + str( ins_ext + len(relations.index)), end='')
        
        requests = requests + 1
        
        #Sleep
        #if requests % 10 == 0:
        #    time.sleep(medium_sleep)
            
        #if requests % 100 == 0:
        #    time.sleep(long_sleep)
        
        if requests % 200 == 0:
            time.sleep(long_sleep)
    

    print('\n')
    updateUsersTable(users_ext_file, users_file)
    if process == 'followers' or process == 'followed':
        updateRelationsTable(relationstrk_file, relations_file)    

    return requests

# Update Users Table

In [11]:
def updateUsersTable(users_ext_file, users_file):
    # This function updates users table (stored in users_file) with data extracted (stored in users_ext_file) 
    # INPUT
    # users_ext_file: csv file with data of users extracted
    # users_file: csv file with data of users
    # OUTPUT
    # No output but users_file is updated and users_ext_file removed
    

       #'userid', 'username', 'full_name', 'profile_pic_url', 'is_private',
       #'is_verified', 'last_update_time', 'is_business_account', 'posts',
       #'followers', 'followed', 'highlighted_stories', 'igtv', 'collections',
       #'connected_fb_page', 'biography', 'is_joined_recently',
       #'business_category_name', 'overall_category_name', 'category_enum',
       #'has_ar_effects', 'has_clips', 'has_guides', 'has_channel',
       #'external_url', 'profile_pic_url_hd', 'external_url_linkshimmed',
       #'inserted_time'
    
    #Define blocks of data to be updated independently
    pk = list(['userid', 'username'])
    basic = list(["full_name", "profile_pic_url", "is_private", "is_verified", "last_update_time_basic"])
    userinfo = list(['is_business_account', 'posts','followers', 'followed', 'highlighted_stories', 'igtv', 'collections',
                     'connected_fb_page', 'biography', 'is_joined_recently','business_category_name', 'overall_category_name', 
                     'category_enum','has_ar_effects', 'has_clips', 'has_guides', 'has_channel','external_url', 'profile_pic_url_hd', 
                     'external_url_linkshimmed', "last_update_time_info"])
    
    blocks = pd.DataFrame({'block_name': ['basic', 'userinfo'],
                           'lastupdatetime': ['last_update_time_basic', 'last_update_time_info'],
                           'block_columns': [basic, userinfo]
                         })

    #If both CSV files exist, then concatenate, deduplicate and remove user_ext
    if os.path.isfile(users_ext_file):
        users_ext = pd.read_csv(users_ext_file, dtype=usercolumntypes)
        print('Loaded file users_ext.csv with ' + str(len(users_ext.index)) + ' records.')
        if os.path.isfile(users_file):
            users = pd.read_csv(users_file, dtype=usercolumntypes)
            print('Loaded file users.csv with ' + str(len(users.index))  + ' records.')
        else:
            users = pd.DataFrame([],columns = pk + basic + userinfo)

        #merge both dfs
        users_ext['last_update_time_basic'] = users_ext['last_update_time']
        users_ext['last_update_time_info'] = users_ext['inserted_time']
        users = pd.concat([users_ext, users])
        
        
        
        #Create finaldf
        final_df = users[pk].drop_duplicates()

        #Lets update each block of informations
        for i in range(len(blocks)):
            
            #get variables from this block
            block_name = blocks["block_name"].iloc[i]
            lastupdatetime = blocks["lastupdatetime"].iloc[i]
            block_columns = blocks["block_columns"].iloc[i]
            
            #Take most updated row for each userid
            sub_df = users[pk + block_columns]
            sub_df = sub_df.sort_values(by = lastupdatetime, ascending=False)
            sub_df = sub_df.drop_duplicates(subset = pk, keep='first')
            
            #merge to final_df
            final_df = pd.merge(final_df, sub_df, on = pk, how='left')

        #remove updated file and overwrite users
        print('Table users updated, now it has ' + str(len(final_df.index)) + ' records.')
        final_df.to_csv(users_file, index = False)
        print('Saved csv file: ' + users_file)
        os.remove(users_ext_file)
        print('Removed csv file: ' + users_ext_file)



# Update Relations Table

In [2]:
def updateRelationsTable(input_trk_file, output_dim_file):
    # This function updates users table (stored in users_file) with data extracted (stored in users_ext_file) 
    # INPUT
    # users_ext_file: csv file with data of users extracted
    # users_file: csv file with data of users
    # OUTPUT
    # No output but users_file is updated and users_ext_file removed
    
    #If both CSV files exist, then concatenate, deduplicate and remove user_ext
    if os.path.isfile(input_trk_file):
        ext = pd.read_csv(input_trk_file)
        print('Loaded file ' + input_trk_file + ' with ' + str(len(ext.index)) + ' records.')
        ext.loc[:,"last_update_time"] = ext.loc[:,"inserted_time"]
        ext.loc[:,"last_following_time"] = ext.loc[:,"inserted_time"]
        
        if os.path.isfile(output_dim_file):
            df = pd.read_csv(output_dim_file)
            print('Loaded file ' + output_dim_file + ' with ' + str(len(df.index))  + ' records.')
            #merge both dfs
            df = pd.concat([ext, df])
        else:
            df = ext

        #Create new dfs
        df_inserted_time = df[["userid", 'followed_by_userid', "inserted_time"]]
        df_lastfwg_time = df[df.status == 'following'][["userid", 'followed_by_userid', "last_following_time"]]


        #Keep last status of each relation
        df = df.sort_values(by="last_update_time", ascending=False) 
        df = df.drop_duplicates(subset=['userid', 'followed_by_userid'], keep='first')
        df = df.drop(['inserted_time','last_following_time'],  axis='columns')
        

        #first-time-seen relation
        df_inserted_time = df_inserted_time.sort_values(by="inserted_time", ascending=True)
        df_inserted_time = df_inserted_time.drop_duplicates(subset=['userid', 'followed_by_userid'], keep='first')
        
        #last-time-following relation
        df_lastfwg_time = df_lastfwg_time.sort_values(by="last_following_time", ascending=False)
        df_lastfwg_time = df_lastfwg_time.drop_duplicates(subset=['userid', 'followed_by_userid'], keep='first')
        
        #merge
        df = pd.merge(df, df_inserted_time, on=['userid','followed_by_userid'], how='left')
        df = pd.merge(df, df_lastfwg_time, on=['userid','followed_by_userid'], how='left')
        #df = df.drop('Unnamed: 0',  axis='columns')
        print('Table relations updated, now it has ' + str(len(df.index)) + ' records.')
        

        #remove updated file and overwrite
        df.to_csv(output_dim_file, index = False)
        print('Saved csv file: ' + output_dim_file)
                    

# Update VipUsers Table

In [3]:
def UpdateVipUsers(user_info):
        
    if os.path.isfile(vipuserstrk_file):
        user_info.to_csv(vipuserstrk_file, mode='a', header=False, index = False) #append
    else:
        user_info.to_csv(vipuserstrk_file, mode='w', header=True , index = False) #create/overwrite
        
    print('Saved csv file: ' + vipuserstrk_file)


# Follow / Unfollow This User

In [1]:
def followUnfollowThisUser(username, process):
    
    status = 'unknown'
    is_done = 0  #boolean to check if the action (follow or unfollow) has been done. 0=false, 1=true, -1=blocked   
    
    url = 'https://www.instagram.com/' + username + '/'
    driver.get(url)
    time.sleep(1)
    try:
      element = driver.find_element_by_xpath("//*[@class='nZSzR']")
    except:
      print('@' + username + ' does not exist anymore. Action ' + process + ' skipped.')
      return is_done
    
    element = element.find_elements_by_tag_name("button")
    button_text = element[0].text

    
    if process == 'follow':
        if button_text.find("Seguir") >= 0: #this includes "Seguir tambien"
            element[0].click()
            time.sleep(2)
            #check if instagram blocked the follow
            element = driver.find_element_by_xpath("//*[@class='nZSzR']")
            element = element.find_elements_by_tag_name("button")
            button_text = element[0].text
            if button_text.find("Seguir") >= 0:
                print("THIS ACTION WAS BLOCKED BY INSTAGRAM <-------------------------------------------")
                print("@" + account + " did not followed @" + username)
                return -1
            status = 'following'
            print("@" + account + " started following @" + username)
            is_done = 1
        else:
            status = 'following'
            print("@" + account + " already follows @" + username + ' (status: ' + button_text + ')')
            
    elif process == 'unfollow':
        if button_text.find("Solicitado") >= 0 or button_text.find("Enviar mensaje") >= 0:
            if button_text.find("Solicitado") >= 0:
                ind = 0
            elif button_text.find("Enviar mensaje") >= 0:
                ind = 1
            element[ind].click()
            unfollow_modal = driver.find_element_by_xpath("//*[@class='mt3GC']")
            unfollow_modal = unfollow_modal.find_elements_by_tag_name("button")
            unfollow_button_text = unfollow_modal[0].text
            time.sleep(1)
            unfollow_modal[0].click()
            status = 'unfollowed'
            print("@" + account + " unfollowed @" + username)
            is_done = 1
        else:
            print("@" + account + " does not follow @" + username + ' (no action required)')
            
    #build output dataframe
    if status != 'unknown':
        users = pd.read_csv(users_file, dtype=usercolumntypes)
        userid = users[users.username == username]["userid"].iloc[0]
        account_id = users[users.username == account]["userid"].iloc[0]

        data = {'status': [status],
                'userid': [userid],
                'username': [username],
                'followed_by_userid': [account_id], 
                'followed_by_username': [account],
                'inserted_time': [datetime.datetime.now()],
                'process':["bot_" + process]
               } 
        relationstrk = pd.DataFrame(data) 
        
        if os.path.isfile(relationstrk_file):
            relationstrk.to_csv(relationstrk_file, mode='a', header=False, index = False) #append
        else:
            relationstrk.to_csv(relationstrk_file, mode='w', header=True , index = False) #create/overwrite

        print('Saved csv file: ' + relationstrk_file)

    return is_done

# Follow/Unfollow list of users

In [9]:
def followUnfollowTheseUsers(username_list, process, max_requests, popularity_ratio):
    # when popularity_ratio (followers/followed) set at 1.5 it means you don't want to follow/unfollow 
    # users with high popularity. 
    
    #Check how many times this process has been performed today
    relationstrk = pd.read_csv(relationstrk_file)
    relations_account = relationstrk[relationstrk.followed_by_username == account]
    actions_account = relations_account[relations_account.process == 'bot_' + process]    
    actions_account['inserted_time'] = actions_account['inserted_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))
    actions_account_6h = actions_account[actions_account.inserted_time > datetime.datetime.now() - datetime.timedelta(hours=6)]
    requests = len(actions_account_6h.iloc[:,0])
    
    #load whithelisted and blacklisted users for this account
    whitelist =  pd.read_csv(users_lists_path + account + '_whitelist') 
    whitelist = list(whitelist["username"])
    blacklist =  pd.read_csv(users_lists_path + account + '_blacklist') 
    blacklist = list(blacklist["username"])
        
    
    for username in username_list:
        
        #initialize parameters
        is_done = 0
        user_followed_before = 0
        is_too_popular = False
        
        # check if number of requests was exceeded
        if requests >= max_requests:
            print('Max_requests exceeded: ' + str(requests))
            break
        
            
        #check if user should skip action
        if process == 'follow':
            user_followed_before = len(relations_account[relations_account.username == username].iloc[:,0])
            is_blacklisted_user = username in blacklist
        elif process == 'unfollow':
            is_whitelisted_user = username in whitelist

        #follow or unfollow the user
        if process == 'follow' and user_followed_before >= 1:
            #print('@' + username + ' was followed somewhen in the past')
            0
        elif process == 'follow' and is_blacklisted_user:
            print('@' + username + ' was not followed since it is in the blacklist')
        elif process == 'unfollow' and is_whitelisted_user:
            print('@' + username + ' was not unfollowed since it is in the whitelist')       
        else:
            if popularity_ratio > 0: 
                #check if account wants to filter by popularity_ratio
                #save_stdout = sys.stdout
                #sys.stdout = open('trash', 'w')
                user_info = UserInfo(username)
                #sys.stdout = save_stdout
                #time.sleep(1)
                user_followers = user_info["followers"][0]
                user_followed = user_info["followed"][0]
                if popularity_ratio < user_followers/user_followed:
                    is_too_popular = True
                    print('@' + username + ' was not ' + process + 'ed since its popularity ratio is too high (' + str(round(user_followers/user_followed,2)) + ')')
                else:
                    is_done = followUnfollowThisUser(username, process)
            else:
                is_done = followUnfollowThisUser(username, process)
 
        
        #if action performed, count up the request
        if is_done == 1:
            requests = requests + 1
            print('Request number ' + str(requests))
            time.sleep(2)
        elif is_done == -1:
            print('The process automatically stopped ' + process + 'ing users since this action has been blocked by instagram')
            print('Total accounts ' + process + 'ed: ' + str(requests))
            break
            
    #before ending update relations dim table        
    updateRelationsTable(relationstrk_file, relations_file)


In [ ]:
print('Libraries and functions loaded correctly ' + str(datetime.datetime.now()))

In [14]:
#REQUEST DE SCROLL DE FOTOS DENTRO DE UN PERFIL, CON INFO BASICA DE LIKES, COMENTARIOS Y NÚMERO DE FOTOS DE LA PUBLICACION

#Este ejemplo es del perfil de influencerdemercadillo
#https://www.instagram.com/graphql/query/?query_hash=003056d32c2554def87228bc3fd9668a&variables=%7B%22id%22%3A%22206189039%22%2C%22first%22%3A12%2C%22after%22%3A%22QVFCbzRNVld4NnpvYzVHbFg2aVRQQ0J6Q3k1RVFoSGV6OGtVMWdqdFZmaTcxNGtleFlENDJ2Q3dBUVlyS3oyU3ZiSUh5UE5BaXVmWWxkdDBLeDd6VGVZNA%3D%3D%22%7D

In [ ]:
#Reconocimiento de imágenes (limites -> solo fans o muestreo aleatorio)
# https://cloud.google.com/vision/                                                                      # 1000 per month
# https://rapidapi.com/imagga/api/imagga-automated-image-tagging-and-categorization                     # 2000 per month
# https://rapidapi.com/everypixel/api/everypixel-image-recognition                                      #  100 per day
# https://rapidapi.com/microsoft-azure-org-microsoft-cognitive-services/api/microsoft-computer-vision3  # 5000 per month


#buscador de apis
# https://rapidapi.com/collection/top-image-recognition-apis